### Building a simple LLM application with LCEL
- Will translate text from english to other languages
- it needs LLM call and prompt with it
- LCEL is used to chain components together 
- will use prompt template , Output parser , LCEL , langsmith and langserve to deploy 

In [3]:
### Open AI API key and open source models -LLama3 , Gemma2 , mistral -- Groq
## pip install langchain_groq , langchain_core

import os 
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [11]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019247803140>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000192478019A0>, model_name='Gemma2-9b-It')

In [12]:
from langchain_core.messages import HumanMessage,SystemMessage

messages = [
    SystemMessage(content="Translate the following from english to french"),
    HumanMessage(content="Hello How are you ?")
]

result = model.invoke(messages)

In [13]:
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family.)\n* **Coucou, ça va ?** (This is a very casual way to say it, like "Hey, what\'s up?")\n\n\nLet me know if you have any other phrases you\'d like translated! \n', response_metadata={'token_usage': {'completion_tokens': 118, 'prompt_tokens': 21, 'total_tokens': 139, 'completion_time': 0.214545455, 'prompt_time': 0.000186599, 'queue_time': 0.013120181, 'total_time': 0.214732054}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1117216-8dff-4682-bbbb-2c430393fbbd-0', usage_metadata={'input_tokens': 21, 'output_tokens': 118, 'total_tokens': 139})

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (This is used with friends and family.)\n* **Coucou, ça va ?** (This is a very casual way to say it, like "Hey, what\'s up?")\n\n\nLet me know if you have any other phrases you\'d like translated! \n'

In [15]:
### Using LCEL - Chain the components

chain = model | parser

response = chain.invoke(messages)

In [16]:
response

'Here\'s the translation of "Hello, how are you?" in French:\n\n**Salut, comment vas-tu ?** \n\n* **Salut** is a casual way to say "Hello".\n* **Comment vas-tu ?** is the most common way to ask "How are you?" in French when speaking to one person. \n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [22]:
### Using Prompt Templates instead of Messages

from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into the following {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",generic_template),
        ("user","{text}")
    ]
)

prompt = prompt_template.invoke({"language":"marathi","text":"Hello ?"})


In [23]:
prompt.to_messages()

[SystemMessage(content='Translate the following into the following marathi:'),
 HumanMessage(content='Hello ?')]

In [25]:
chain = prompt_template | model | parser
response = chain.invoke({"language":"marathi","text":"Hello"})

In [26]:
response

'नमस्कार (Namaskar) \n'